# Analysis 16: Organize Candidate Genes

In [ ]:

library(dplyr)



Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Attaching package: 'data.table'

The following objects are masked from 'package:dplyr':

    between, first, last

...

The following object is masked from 'package:data.table':

    shift

The following objects are masked from 'package:dplyr':

    collapse, desc, slice


Attaching package: 'AnnotationDbi'

The following object is masked from 'package:clusterProfiler':

    select

The following object is masked from 'package:dplyr':

    select

# Inputs

`candidate_out_name` - path to the candidate file output by `analysis/call_candidate_genes.qmd` and then modified in `analysis/candidate_gene_effects.qmd` `tox_meta_fn` - path to the toxin metadata file `ranked_qtl_fn` - path to the ranked QTL data output by `analysis/rank_regions.qmd`

In [ ]:

candidate_out_name <- "data/processed/interval_genes/candidate_genes/inbred_candidate_genes_with_betas.tsv.gz"
tox_meta_fn <- "data/processed/tox_data/tox_metadata.csv"

# Path to ranked QTL data
ranked_qtl_fn <- "data/processed/qtl_ranking/inbred/ranked_qtl.tsv"


# Functions

In [ ]:
# Function to pull the max log10 p-value for each candidate gene
#' @param candidate_raw output of the finemapping.curation.R script
#' @return A data frame with the min p-value for each candidate gene and associated marker information
max_log10p_candidates <- function(candidate_raw) {
  # pull out the min p-value for each candidate gene and keep the associated marker info
  gene_max <- candidate_raw %>%
    dplyr::group_by(WBGeneID) %>%
    dplyr::arrange(desc(VARIANT_LOG10p)) %>%
    dplyr::slice(1) %>%
    dplyr::select(
      WBGeneID,
      max_log10p = VARIANT_LOG10p,
      max_marker = MARKER,
      max_marker_ld = VARIANT_LD_WITH_PEAK_MARKER,
      max_marker_maf = MAF_variant,
      beta_coefficient = beta_coefficient,
      effect_allele = effect_allele,
      other_allele = other_allele
    ) %>%
    dplyr::ungroup()

  return(gene_max)
}


# Main

In [ ]:

## Process the Tox data ##
# load the toxicant condition metadata
tox_meta <- data.table::fread(tox_meta_fn)

# load the ranked QTL data
ranked_qtl <- data.table::fread(ranked_qtl_fn) %>%
  dplyr::select(trait, marker, rank)

## Read in the candidate gene data ##
# Read in the candidate gene table
candidate_raw <- data.table::fread(candidate_out_name)

# format the peakmarker column of the candidate gene table to have correct naming syntax

candidate_anno <- candidate_raw %>%
  dplyr::mutate(
    PEAK_MARKER = sapply(
      PEAK_MARKER,
      convert_marker,
      to_marker = "RM",
      from_marker = "NUM"
    ),
    MARKER = stringr::str_replace_all(
      MARKER,
      "_",
      ":"
    )
  ) %>%
  dplyr::left_join(
    tox_meta,
    by = c("trait")
  )

candidate_anno_rank <- left_join(
  candidate_anno,
  ranked_qtl,
  by = c(
    "trait" = "trait",
    "PEAK_MARKER" = "marker"
  )
)

## Convert WBid to readable gene

# pull the WORMBASE ID from the candidate gene table
wormbase_id <- candidate_anno_rank %>%
  dplyr::pull(WBGeneID)

ids <- clusterProfiler::bitr(
  wormbase_id,
  fromType = "WORMBASE",
  toType = c("GENENAME", "SYMBOL"),
  OrgDb = "org.Ce.eg.db"
)


'select()' returned 1:1 mapping between keys and columns

Look at the number of candidate genes

In [ ]:
n_candidates <- nrow(candidate_flex_table_pre)
n_candidates


[1] 136

# Save the candidate gene table

In [ ]:
# Path to save the candidate gene table
candidate_table_fn <- "data/processed/interval_genes/candidate_genes/inbred_candidate_genes_table.tsv"

# Save the candidate gene table
data.table::fwrite(
  candidate_flex_table_pre,
  file = candidate_table_fn,
  sep = "\t",
  na = "NA",
  quote = FALSE
)
